In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [2]:
df = pd.read_csv('./DS-Notebooks-Summer-2022/week1/Data/avocado.csv')

In [3]:
df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.drop(['Unnamed: 0', '4046', '4225', '4770','year'], axis=1, inplace=True)
df.head()

,Date,AveragePrice,Total Volume,Total Bags,Small Bags,Large Bags,XLarge Bags,type,region
0,2015-12-27,1.33,64236.62,8696.87,8603.62,93.25,0.0,conventional,Albany
1,2015-12-20,1.35,54876.98,9505.56,9408.07,97.49,0.0,conventional,Albany
2,2015-12-13,0.93,118220.22,8145.35,8042.21,103.14,0.0,conventional,Albany
3,2015-12-06,1.08,78992.15,5811.16,5677.40,133.76,0.0,conventional,Albany
4,2015-11-29,1.28,51039.60,6183.95,5986.26,197.69,0.0,conventional,Albany


In [5]:
df['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

In [6]:
df.columns = df.columns.str.replace(' ','_')
df.head()

,Date,AveragePrice,Total_Volume,Total_Bags,Small_Bags,Large_Bags,XLarge_Bags,type,region
0,2015-12-27,1.33,64236.62,8696.87,8603.62,93.25,0.0,conventional,Albany
1,2015-12-20,1.35,54876.98,9505.56,9408.07,97.49,0.0,conventional,Albany
2,2015-12-13,0.93,118220.22,8145.35,8042.21,103.14,0.0,conventional,Albany
3,2015-12-06,1.08,78992.15,5811.16,5677.40,133.76,0.0,conventional,Albany
4,2015-11-29,1.28,51039.60,6183.95,5986.26,197.69,0.0,conventional,Albany


In [7]:
df.dtypes

Date             object
AveragePrice    float64
Total_Volume    float64
Total_Bags      float64
Small_Bags      float64
Large_Bags      float64
XLarge_Bags     float64
type             object
region           object
dtype: object

In [8]:
X=df.drop('AveragePrice',axis=1).copy()
X.head()

,Date,Total_Volume,Total_Bags,Small_Bags,Large_Bags,XLarge_Bags,type,region
0,2015-12-27,64236.62,8696.87,8603.62,93.25,0.0,conventional,Albany
1,2015-12-20,54876.98,9505.56,9408.07,97.49,0.0,conventional,Albany
2,2015-12-13,118220.22,8145.35,8042.21,103.14,0.0,conventional,Albany
3,2015-12-06,78992.15,5811.16,5677.40,133.76,0.0,conventional,Albany
4,2015-11-29,51039.60,6183.95,5986.26,197.69,0.0,conventional,Albany


In [9]:
y=df['AveragePrice'].copy()
y.head()

0    1.33
1    1.35
2    0.93
3    1.08
4    1.28
Name: AveragePrice, dtype: float64

In [10]:
X.dtypes

Date             object
Total_Volume    float64
Total_Bags      float64
Small_Bags      float64
Large_Bags      float64
XLarge_Bags     float64
type             object
region           object
dtype: object

In [11]:
pd.get_dummies(X, columns=['Date']).head()

,Total_Volume,Total_Bags,Small_Bags,Large_Bags,XLarge_Bags,type,region,Date_2015-01-04,Date_2015-01-11,Date_2015-01-18,...,Date_2018-01-21,Date_2018-01-28,Date_2018-02-04,Date_2018-02-11,Date_2018-02-18,Date_2018-02-25,Date_2018-03-04,Date_2018-03-11,Date_2018-03-18,Date_2018-03-25
0,64236.62,8696.87,8603.62,93.25,0.0,conventional,Albany,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,54876.98,9505.56,9408.07,97.49,0.0,conventional,Albany,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,118220.22,8145.35,8042.21,103.14,0.0,conventional,Albany,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,78992.15,5811.16,5677.40,133.76,0.0,conventional,Albany,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51039.60,6183.95,5986.26,197.69,0.0,conventional,Albany,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X_encoded = pd.get_dummies(X, columns=['type'])
X_encoded.head()

,Date,Total_Volume,Total_Bags,Small_Bags,Large_Bags,XLarge_Bags,region,type_conventional,type_organic
0,2015-12-27,64236.62,8696.87,8603.62,93.25,0.0,Albany,1,0
1,2015-12-20,54876.98,9505.56,9408.07,97.49,0.0,Albany,1,0
2,2015-12-13,118220.22,8145.35,8042.21,103.14,0.0,Albany,1,0
3,2015-12-06,78992.15,5811.16,5677.40,133.76,0.0,Albany,1,0
4,2015-11-29,51039.60,6183.95,5986.26,197.69,0.0,Albany,1,0


In [13]:
y.unique()

array([1.33, 1.35, 0.93, 1.08, 1.28, 1.26, 0.99, 0.98, 1.02, 1.07, 1.12,
       1.31, 1.11, 1.34, 1.45, 1.05, 1.37, 1.27, 1.32, 1.23, 1.19, 1.43,
       1.2 , 1.22, 1.13, 1.16, 1.06, 1.17, 1.24, 0.96, 1.  , 1.09, 0.95,
       0.97, 1.15, 1.01, 1.14, 1.1 , 1.04, 1.03, 1.18, 1.29, 1.25, 0.71,
       0.94, 0.91, 1.39, 1.4 , 1.36, 1.3 , 1.54, 1.59, 1.56, 1.38, 1.49,
       1.5 , 1.47, 1.42, 1.41, 1.44, 1.52, 0.9 , 0.87, 0.78, 0.92, 0.83,
       0.88, 0.84, 0.85, 1.21, 0.79, 0.89, 0.86, 0.8 , 0.82, 0.74, 0.81,
       0.77, 0.75, 0.65, 0.73, 0.68, 0.72, 0.67, 0.76, 0.62, 0.7 , 0.66,
       1.46, 0.49, 0.53, 0.56, 0.6 , 0.58, 0.54, 0.52, 0.51, 0.57, 0.63,
       0.64, 0.61, 1.48, 1.55, 1.58, 1.53, 1.57, 1.51, 0.69, 1.62, 1.63,
       1.61, 1.6 , 1.73, 1.8 , 1.86, 1.85, 1.69, 1.66, 1.65, 1.64, 1.67,
       1.68, 1.82, 2.07, 1.84, 1.83, 1.7 , 1.72, 1.81, 1.78, 0.59, 1.9 ,
       1.93, 1.99, 1.95, 1.89, 1.76, 1.71, 1.74, 0.55, 1.79, 1.75, 1.94,
       2.2 , 1.87, 1.77, 2.06, 2.11, 2.22, 2.14, 2.

In [14]:
sum(y)/len(y)

1.4059784097758825

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X_encoded, y, random_state=42)

In [16]:
len(X_train)

13686

In [17]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',missing=None,seed=42)
clf_xgb.fit(X_train,y_train,verbose=True,early_stopping_rounds=10,eval_metric="aucpr",eval_set=[(X_test,y_test)])

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 